In [18]:
import matplotlib.pyplot as plt
%matplotlib tk 
import numpy as np
from scipy.constants import au
from scipy.optimize import minimize
from poliastro.bodies import Sun
from poliastro.core.iod import izzo
from poliastro.twobody import Orbit
from astropy import units as u
from astropy.time import Time
from datetime import datetime

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates

In [20]:
targets = ['SUN', 'EARTH', 'MARS', 'SATELLITE']

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.set_xlabel('X (AU)')
ax.set_ylabel('Y (AU)')
ax.set_zlabel('Z (AU)')
ax.set_title('Orbit in AU Space')

all_px, all_py, all_pz = [], [], []

for target in targets:
    try :
        data = np.genfromtxt(f'simulation_results/{target}.csv', delimiter=',', skip_header=1)
    except :
        data = np.genfromtxt(f'traces/{target}.csv', delimiter=',', skip_header=1)
        
    px, py, pz = data[:, 1], data[:, 2], data[:, 3]
    
    all_px.extend(px)
    all_py.extend(py)
    all_pz.extend(pz)
    
    ax.plot(px, py, pz, label=target)  # Add label for better visualization

# Compute overall range
all_px, all_py, all_pz = np.array(all_px), np.array(all_py), np.array(all_pz)

max_range = max(all_px.ptp(), all_py.ptp(), all_pz.ptp()) / 2.0
mid_x = (all_px.max() + all_px.min()) / 2.0
mid_y = (all_py.max() + all_py.min()) / 2.0
mid_z = (all_pz.max() + all_pz.min()) / 2.0

# Set the global limits
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

ax.legend()  # Show labels

#plt.show()
print('done')

3

In [21]:
def load_data(host : str, target : str) :
    
    try:
        
        dates = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[0],skip_header=True,dtype=str) 

        host_positions = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        host_velocities = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)

        target_positions = np.genfromtxt(f'simulation_results\{target}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        target_velocities = np.genfromtxt(f'simulation_results\{target}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)  
    
    except : 
        
        dates = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[0],skip_header=True,dtype=str) 

        host_positions = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        host_velocities = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)

        target_positions = np.genfromtxt(f'traces\{target}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        target_velocities = np.genfromtxt(f'traces\{target}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)  
        
    return dates, host_positions, host_velocities, target_positions, target_velocities

In [26]:
dates, host_positions, host_velocities, target_positions, target_velocities = load_data('EARTH_TRACE','MARS_TRACE')
elements = dates.shape[0] 
mission_durations = np.arange(20,140,1)


In [ ]:
for i in range(0,elements-1000,100) :
    for j in range()
    v_i,v_f = izzo(Sun.k,host_positions[i],target_positions[i+1000],50*24*60*60,0,True,True,5,1e-3)